## Example

In [3]:
import sys
sys.path.append("../")

from cv_expt.base.data.base_dataset import ImageDataset, ImageDataType
from cv_expt.base.defs.defs import ImageChannel, DataType
from cv_expt.base.logger.base_logger import BaseLogger, BaseLoggerConfig
from cv_expt.base.configs.configs import TrainerConfig, DataConfig
from cv_expt.vis.visualization import subplot_images
from cv_expt.base.logger.base_logger import BaseLogger
from cv_expt.base.models.base_model import ModelWrapper
from cv_expt.base.trainer.base_trainer import Trainer

from pathlib import Path
from typing import Optional
import torch
import cv2
import numpy as np
from tqdm import tqdm  # for progress bar
from torch import nn
from torch.optim import Optimizer
import segmentation_models_pytorch as smp
import datetime

data_config = DataConfig(
    data_path=Path("../data/coco2017"),
    image_channels=ImageChannel.RGB,
    image_extensions=["jpg", "png"],
    max_data=-1,
    samples_per_epoch=2000,
)
train_dataset = ImageDataset(
    data_config,
    return_type=ImageDataType.TENSOR,
    normalization=lambda x: x / 255.0,
    denormalization=lambda x: (x * 255.0),
)
data_config.samples_per_epoch = 100
val_dataset = ImageDataset(
    data_config,
    data_type=DataType.VALIDATION,
    return_type=ImageDataType.TENSOR,
    normalization=lambda x: x / 255.0,
    denormalization=lambda x: (x * 255.0),
)

config = TrainerConfig(
    result_dir=Path("../results"),
    expt_name="expt1",
    run_name=f"run_{datetime.datetime.now().date()}",
    epochs=100,
    show_images=True,
)
model = smp.Unet("resnet18", classes=3, in_channels=1, activation="sigmoid")
model = ModelWrapper(
    model,
    postprocess=val_dataset.denormalization,
    preprocess=val_dataset.normalization,
)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()
trainer = Trainer(
    model,
    config,
    optimizer,
    criterion,
    BaseLogger(BaseLoggerConfig(log_path=config.result_dir, name='gray2rgb')),
    train_dataset,
    val_dataset,
)
trainer.train()

Data type: DataType.TRAIN, Number of images: 131165


## Inference of Some Unseen Images

In [ ]:
!which python

/home/blackphoenix/projects/computer-vision-experiments/venv/bin/python


In [ ]:
!pip install -e /home/blackphoenix/projects/computer-vision-experiments/

Obtaining file:///home/blackphoenix/projects/computer-vision-experiments
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cv_expt (pyproject.toml) ... done
  Created wheel for cv_expt: filename=cv_expt-0.1-0.editable-py3-none-any.whl size=9077 sha256=25d82140553947abd2ba2a77463ae332c7e4fc04ef0511540fdb073755cd2361
  Stored in directory: /tmp/pip-ephem-wheel-cache-9p5eo7wf/wheels/ab/0d/cf/3387bf30d53f89e8cd19fd9f391bcd3fb9d310b6f0f9dcba94
Successfully built cv_expt
  Attempting uninstall: cv_expt
    Found existing installation: cv_expt 0.1
    Uninstalling cv_expt-0.1:
      Successfully uninstalled cv_expt-0.1

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


/home/blackphoenix/projects/computer-vision-experiments/notebooks


ModuleNotFoundError: No module named 'cv_expt'